### 查看数据

In [10]:
import pandas as pd

df_user= pd.read_csv('JD_data/JData_User.csv')
display(df_user.head())

df_month3 = pd.read_csv('JD_data/JData_Action_201603.csv')
display(df_month3.head())

,user_id,age,sex,user_lv_cd,user_reg_tm
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26


,user_id,sku_id,time,model_id,type,cate,brand
0,280567,167208,2016-02-29 23:59:01,0.0,6,4,519
1,270248,35533,2016-02-29 23:59:02,111.0,6,4,306
2,203360,78694,2016-02-29 23:59:02,NaN,1,8,244
3,252369,90402,2016-02-29 23:59:03,0.0,6,7,38
4,279590,154208,2016-02-29 23:59:03,0.0,6,5,570


In [11]:
df_month3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25916378 entries, 0 to 25916377
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   int64  
 1   sku_id    int64  
 2   time      object 
 3   model_id  float64
 4   type      int64  
 5   cate      int64  
 6   brand     int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 1.4+ GB


In [12]:
# 垃圾回收，避免缓存的数据量过大而崩溃
import gc
del df_user
del df_month3
gc.collect()

19554

In [14]:
# 利用pd.merge根据user_id连接数据，判断是否减少：
def user_action_id_check():
    df_user = pd.read_csv('JD_data/JData_User.csv')
    df_user = df_user.loc[:,'user_id'].to_frame()
    df_month2 = pd.read_csv('JD_data/JData_Action_201602.csv')
    print('Is action of Feb.from User file?',len(df_month2) == len(pd.merge(df_user,df_month2)))
    df_month3 = pd.read_csv('JD_data/JData_Action_201603.csv')
    print('Is action of Mar.from User file?',len(df_month3) == len(pd.merge(df_user,df_month3)))
    df_month4 = pd.read_csv('JD_data/JData_Action_201604.csv')
    print('Is action of Apr.from User file?',len(df_month4) == len(pd.merge(df_user,df_month4)))
    del df_user,df_month2,df_month3,df_month4
    gc.collect()
user_action_id_check()

Is action of Feb.from User file? True
Is action of Mar.from User file? True
Is action of Apr.from User file? True


In [15]:
# 检查是否有重复数据
def deduplicate(filepath,filename):
    df_file = pd.read_csv(filepath)
    
    before = df_file.shape[0] # 原文件总数据量
    df_file.drop_duplicates(inplace=True) # 去重
    after = df_file.shape[0] #去重后，数据量
    n_dup = before - after
    if n_dup != 0:
        print('NO .of duplicate records for ' + filename + 'is:' + str(n_dup))
    else:
        print('NO duplicate records in ' + filename)
    del df_file
    gc.collect()

In [16]:
%%time
deduplicate('JD_data/JData_Action_201602.csv','Feb. action')
deduplicate('JD_data/JData_Action_201603.csv','Mar. action')
deduplicate('JD_data/JData_Action_201604.csv','Apr. action')
deduplicate('JD_data/JData_Comment.csv','Comment')
deduplicate('JD_data/JData_Product.csv','Product')
deduplicate('JD_data/JData_User.csv','User')

NO .of duplicate records for Feb. actionis:2756093
NO .of duplicate records for Mar. actionis:7085037
NO .of duplicate records for Apr. actionis:3672710
NO duplicate records in Comment
NO duplicate records in Product
NO duplicate records in User
CPU times: total: 11.1 s
Wall time: 32.6 s


In [18]:
df_month3 = pd.read_csv('JD_data/JData_Action_201603.csv')
IsDuplicated = df_month3.duplicated() # boolean 重复，True,不重复，False

df_d = df_month3[IsDuplicated] #取重复数据
display(df_d.groupby('type').count()) #统计计数

del df_month3,df_d
gc.collect()

,user_id,sku_id,time,model_id,cate,brand
type,,,,,,
1,4659616,4659616,4659616,0,4659616,4659616
2,1406,1406,1406,0,1406,1406
3,2360,2360,2360,0,2360,2360
4,39,39,39,0,39,39
5,4475,4475,4475,0,4475,4475
6,2417141,2417141,2417141,2355765,2417141,2417141


50564

结论发现，重复数据大多数都是由于浏览(1),或者点击(6)产生

重复数据是有意义的，比如用户同时购买多件商品，同时添加多个数量的商品到购物车等